<a href="https://colab.research.google.com/github/KathleenYager/Amazon_Vine_Analysis/blob/main/Vine_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-26 00:01:39--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-06-26 00:01:40 (6.40 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!|2015-08-31 00:00:00|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...| 

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
from pyspark.sql.functions import to_date
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", "yyyy-MM-dd").alias("review_date")]).drop_duplicates()
review_id_df.show()


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100I5J3RRZ11O|   25900467|B00CZ21AJG|     864194424| 2015-02-27|
|R100R05QSIL612|   36538264|B001V9ACMK|     476749154| 2014-01-03|
|R100TZ1EC98UWW|   35574573|B00008W72D|      39700156| 2014-11-07|
|R100UUOG2DE7XW|    3001255|B00IK6AI5I|     912222041| 2014-08-29|
|R100YEWZT398BA|    5182848|B00F4LYUMO|     938068194| 2015-07-20|
| R1018NS1IXQSP|   48845208|B002IPGXVA|     502904590| 2015-01-25|
|R1019B0Y3HPOBW|   47595692|B0084VB54W|      82332203| 2013-09-05|
|R101G39LBIPJBD|   50339089|B00DK25JMO|     529704331| 2015-01-31|
|R101X0FUM5WDCZ|   24146893|B004T8JWA8|     881882087| 2015-02-13|
|R102JT4CNLWNHA|   42515462|B008QPHB30|     841862212| 2013-01-10|
|R1032TWY3L37UM|   43980953|B00005O0EU|     812312514| 2008-10-08|
|R10370H6JTGUFK|   16105646|B001KPMG06|     715905213| 2014-06

In [6]:
# Create the customers_table DataFrame
# customers_df = df.groupby("").agg({""}).withColumnRenamed("", "customer_count")
customers_df = df.groupBy("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   16989307|             1|
|   45632184|             2|
|   14703850|            13|
|   49645387|             2|
|   16343477|             1|
|   15554899|             1|
|   17067926|             1|
|   50843047|             2|
|    4051424|             1|
|   11487525|             1|
|   19371753|             1|
|   18634862|             1|
|   14552054|             1|
|   52695798|             1|
|   49438424|             3|
|   10854449|             9|
|   48521319|             1|
|   11839424|             2|
|   27887950|             1|
|   45392827|             3|
+-----------+--------------+
only showing top 20 rows



In [7]:
# Create the products_table DataFrame and drop duplicates. 
# products_df = df.select([]).drop_duplicates()
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00IANTUPU|Lego, Star Wars, ...|
|B002TMK962|Rock - Concert Ti...|
|B00AOTTSQY|Adult & Children ...|
|B00CH3O5S6|48 Wholesale Wood...|
|B00MF3SAF2|Teenitor 3.7V 680...|
|B004D0C5WY|ThinkMax Silver B...|
|B00NHQHJZS|LEGO Technic Arct...|
|B00NQJ2DBG|Super Bright Long...|
|B0082IGA06|Yu-Gi-Oh! - Dark ...|
|1609581520|American Girl Car...|
|B004RGYVLC|DC Comics POP Her...|
|B0053EX3D4|Inuyasha Kirara T...|
|B00OTLHNI8|Bloco Toys T-Rex ...|
|B00BXEYI8W|Kid Galaxy Robot ...|
|B00JUG2NSM|Weiss Schwarz Kil...|
|B00LVPY252|Maxx Action 30" T...|
|B00IGRHWSC|Mattel Hot Wheels...|
|B00U6FU3YM|Educational Insig...|
|B00Z7FS3Q6|Beach Ball Set, 3...|
|B000WEAGTY|Inflatable Tradit...|
+----------+--------------------+
only showing top 20 rows



In [8]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

In [9]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RDIJS7QYB6XNR|          5|            0|          0|   N|                Y|
|R36ED1U38IELG8|          5|            0|          0|   N|                Y|
| R1UE3RPRGCOLD|          2|            1|          1|   N|                Y|
|R298788GS6I901|          5|            0|          0|   N|                Y|
|  RNX4EXOBBPN5|          1|            1|          1|   N|                Y|
|R3BPETL222LMIM|          5|            0|          0|   N|                Y|
|R3SORMPJZO3F2J|          3|            2|          2|   N|                Y|
|R2RDOJQ0WBZCF6|          5|            0|          0|   N|                Y|
|R2B8VBEPB4YEZ7|          5|            0|          0|   N|                Y|
|R1CB783I7B0U52|          1|            0|          1|   N|     

In [10]:
total_votes_df = vine_df.filter("total_votes>20").select("review_id", "total_votes", "helpful_votes", "vine", "star_rating")
total_votes_df.show(5)


+--------------+-----------+-------------+----+-----------+
|     review_id|total_votes|helpful_votes|vine|star_rating|
+--------------+-----------+-------------+----+-----------+
| ROP6ITXO8K5V2|         27|           23|   N|          5|
|R3ND1LVU7AXCVF|         21|           21|   N|          1|
|R1QS8AOD6HX3ED|         81|           59|   N|          4|
|R3ED60RC69CCQ6|         23|           22|   Y|          5|
|R2MDI1TP46VSYD|         21|            9|   N|          3|
+--------------+-----------+-------------+----+-----------+
only showing top 5 rows



In [11]:
total_votes_df.count()


65871

In [12]:
helpful_votes_df = total_votes_df.filter(total_votes_df["helpful_votes"]/total_votes_df["total_votes"] >=.50)
helpful_votes_df.show(5)

+--------------+-----------+-------------+----+-----------+
|     review_id|total_votes|helpful_votes|vine|star_rating|
+--------------+-----------+-------------+----+-----------+
| ROP6ITXO8K5V2|         27|           23|   N|          5|
|R3ND1LVU7AXCVF|         21|           21|   N|          1|
|R1QS8AOD6HX3ED|         81|           59|   N|          4|
|R3ED60RC69CCQ6|         23|           22|   Y|          5|
|R2JM687C525WR9|         33|           33|   Y|          3|
+--------------+-----------+-------------+----+-----------+
only showing top 5 rows



In [27]:
vine_total_df = helpful_votes_df.filter(helpful_votes_df["star_rating"]>= "5").count()
print (vine_total_df)

28453


In [28]:
vine_star_y_rating_df = vine_total_y_df.filter(vine_total_y_df["star_rating"] == "5").count()
print(vine_star_y_rating_df)




410


In [35]:
percent_star_y_rating = (vine_star_y_rating_df/vine_total_df)
print(percent_star_y_rating)

0.014409728323902577


In [38]:
vine_total_n_df = helpful_votes_df.filter(helpful_votes_df["vine"]=="N")
vine_total_n_df.show()

+--------------+-----------+-------------+----+-----------+
|     review_id|total_votes|helpful_votes|vine|star_rating|
+--------------+-----------+-------------+----+-----------+
| ROP6ITXO8K5V2|         27|           23|   N|          5|
|R3ND1LVU7AXCVF|         21|           21|   N|          1|
|R1QS8AOD6HX3ED|         81|           59|   N|          4|
|R2LWX4TZ67FWPT|         50|           50|   N|          4|
|R29IYHPYD14AGI|         84|           84|   N|          3|
| RRO8C1IVZMD6H|         22|           22|   N|          5|
|R2GHOWIWDVLN18|         36|           28|   N|          1|
|R30PJ9XUHFTY5D|         43|           42|   N|          5|
|R2F7D9N3SSQC20|         34|           32|   N|          4|
| RN4VK9FV66YPS|         33|           29|   N|          5|
|R18T8SFBDXAKSY|         95|           95|   N|          5|
|R36SKVJXZ0I98A|        127|          124|   N|          5|
|R3PAIHQSO707G4|        221|          189|   N|          5|
|R2TETIV0IVWFSJ|         82|           8

In [39]:
vine_star_n_rating_df = vine_total_n_df.filter(vine_total_n_df["star_rating"] == "5").count()
print(vine_star_n_rating_df)

28043


In [40]:
percent_star_n_rating = (vine_star_n_rating_df/vine_total_df)
print(percent_star_n_rating)

0.9855902716760975
